### ====================================
### LabelEncoder
sklearn.preprocessing.LabelEncoder()
[▼DOC](
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder
)  
> 0 から n_classes-1 の間の値を持つターゲットラベルをエンコードします。  
この変換器は、ターゲット値、すなわち y をエンコードするために使用されるべきであり、  
入力 X ではありません。  

・元の値を整数値に変換する。元々数値でも変換できる。    
・数値として扱えるようになるが、  
　名義特徴量(COLORとか)については数値になったことで順序ができてしまうのがNG。   
・数値のマッピング0,1,2,,,は、元の値の昇順の模様。  
・Nanがあると変換できずにエラーになる  
・Xも変換できるのだろうけど、そもそもYの変換を目的に作られているのかな？  
　たしかにデータフレーム丸ごと変換できない、Nanあるとダメってことからもそんな感じがする。  
　勉強に使ってたテキストとかkaggleコースとかもフツーにXに使ってたけども。  

In [6]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format 

In [27]:
# どこの所属？
try: 
    from sklearn.preprocessing import LabelEncoder
    print("sklearn.preprocessingにある")
except ImportError:
    print("sklearn.preprocessingにはない")

try:
    from category_encoders import LabelEncoder
    print("category_encodersにある")
except ImportError:
    print("category_encodersにはない")

sklearn.preprocessingにある
category_encodersにはない


In [17]:
df = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", "Red", "Blue"],
        "SIZE" : ["S", "M", "L", "S", "S"],
        "WEIGHT" : [200, 100, 300, 300, 300],
        "HARDNESS" : ["hard", "soft", "medium", "hard", "soft"],
        "OUTCOME" : [1,2,0,0,0] })
df

,COLOR,SIZE,WEIGHT,HARDNESS,OUTCOME
0,Red,S,200,hard,1
1,Yellow,M,100,soft,2
2,Red,L,300,medium,0
3,Red,S,300,hard,0
4,Blue,S,300,soft,0


In [18]:
# 説明変数X と 目的変数Yに分ける
df_X = df[["COLOR", "SIZE", "WEIGHT","HARDNESS"]].copy()
df_Y = df[["OUTCOME"]].copy()

In [19]:
df_X_test = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", "Green"],
        "SIZE" : ["S", "M", "L", "XL", ],
        "WEIGHT" : [200, 100, 300, 400],
        "HARDNESS" : ["hard", "soft", "medium", "maxhard"] })
df_X_test

,COLOR,SIZE,WEIGHT,HARDNESS
0,Red,S,200,hard
1,Yellow,M,100,soft
2,Red,L,300,medium
3,Green,XL,400,maxhard


- エンコードしてみる

In [23]:
print("エンコード前")
display(df_X)

print("エンコード後")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 1列ずつSeriesにしてエンコード
df_X_le = pd.DataFrame({
    "COLOR" : le.fit_transform(df_X["COLOR"]),
    "SIZE"  : le.fit_transform(df_X["SIZE"]),
    "WEIGHT"  : le.fit_transform(df_X["WEIGHT"]),
    "HARDNESS"  : le.fit_transform(df_X["HARDNESS"]),
})
display(df_X_le)

エンコード前


,COLOR,SIZE,WEIGHT,HARDNESS
0,Red,S,200,hard
1,Yellow,M,100,soft
2,Red,L,300,medium
3,Red,S,300,hard
4,Blue,S,300,soft


エンコード後


,COLOR,SIZE,WEIGHT,HARDNESS
0,1,2,1,0
1,2,1,0,2
2,1,0,2,1
3,1,2,2,0
4,0,2,2,2


- 並び順の指定

In [24]:
# 数値のマッピングは、元の値の昇順ぽいので
# 自分で指定したいときは、接頭語つけてあげればいい
df_X_tmp = df_X.copy()
df_X_tmp["HARDNESS"] = df_X_tmp["HARDNESS"].mask(df_X_tmp["HARDNESS"]=="hard",   "2_hard")
df_X_tmp["HARDNESS"] = df_X_tmp["HARDNESS"].mask(df_X_tmp["HARDNESS"]=="medium", "1_medium")
df_X_tmp["HARDNESS"] = df_X_tmp["HARDNESS"].mask(df_X_tmp["HARDNESS"]=="soft",   "0_soft")

print("エンコード前")
display(df_X_tmp[["HARDNESS"]])

print("エンコード後")
le = LabelEncoder()

# 1列ずつSeriesにしてエンコード
df_X_le = pd.DataFrame({
    "HARDNESS"  : le.fit_transform(df_X_tmp["HARDNESS"]),
})
display(df_X_le)

エンコード前


,HARDNESS
0,2_hard
1,0_soft
2,1_medium
3,2_hard
4,0_soft


エンコード後


,HARDNESS
0,2
1,0
2,1
3,2
4,0


- Nanがあると変換できない

In [25]:
# Nanがあると変換できない
df_X_tmp = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", np.nan, "Blue"],
        "HARDNESS" : ["hard", np.nan, "medium", "hard", "soft"] })

#print("エンコード前")
#display(df_X_tmp)

#print("エンコード後")
#le = LabelEncoder()

try:
    # 1列ずつSeriesにしてエンコード
    df_X_le = pd.DataFrame({
        "COLOR" : le.fit_transform(df_X_tmp["COLOR"]),
        "HARDNESS"  : le.fit_transform(df_X_tmp["HARDNESS"]),
    })
    display(df_X_le)

except TypeError as e:
    print("エラー:", e)

エラー: argument must be a string or number


- fit時にはなかった値があるとtransfromでエラー

In [26]:
# fit時にはなかった値があるとtransfromでエラー
print("エンコード前")
display(df_X_test["COLOR"])

print("エンコード後")
le = LabelEncoder()

# 1列ずつSeriesにしてエンコード
le.fit(df_X["COLOR"])
try:
    le.transform(df_X_test["COLOR"])
except ValueError as e:
    print("エラー:", e)

エンコード前


0       Red
1    Yellow
2       Red
3     Green
Name: COLOR, dtype: object

エンコード後
エラー: y contains previously unseen labels: 'Green'
